In [1]:
import pandas as pd  
import matplotlib.pyplot as plt
# Charger les fichiers CSV (modifie le nom si nécessaire)
train_df = pd.read_csv("C:\\Users\\joyce\\Desktop\\mesdocuments\\data science\\projet\\Sales-Prediction-ML\\data\\train.csv") 
test_df = pd.read_csv("C:\\Users\\joyce\\Desktop\\mesdocuments\\data science\\projet\\Sales-Prediction-ML\\data\\test.csv")
oil_df = pd.read_csv("C:\\Users\\joyce\\Desktop\\mesdocuments\\data science\\projet\\Sales-Prediction-ML\\data\\oil.csv")
stores_df = pd.read_csv("C:\\Users\\joyce\\Desktop\\mesdocuments\\data science\\projet\\Sales-Prediction-ML\\data\\stores.csv")
transactions_df = pd.read_csv("C:\\Users\\joyce\\Desktop\\mesdocuments\\data science\\projet\\Sales-Prediction-ML\\data\\transactions.csv")
holidays_df = pd.read_csv("C:\\Users\\joyce\\Desktop\\mesdocuments\\data science\\projet\\Sales-Prediction-ML\\data\\holidays_events.csv")

oil_df['dcoilwtico'] = oil_df['dcoilwtico'].fillna(oil_df['dcoilwtico'].median())

train_df = train_df.drop_duplicates()

train_df['date'] = pd.to_datetime(train_df['date'])
test_df['date'] = pd.to_datetime(test_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'])
holidays_df['date'] = pd.to_datetime(holidays_df['date'])

train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['day_of_week'] = train_df['date'].dt.dayofweek  

In [11]:
#Encodage des variables catégorielles

In [9]:
from sklearn.preprocessing import LabelEncoder

# Encodage de 'store_type' dans stores.csv
label_encoder = LabelEncoder()
stores_df['store_type_encoded'] = label_encoder.fit_transform(stores_df['type'])

# Fusionner avec train.csv en utilisant 'store_nbr' comme clé
train_df= train_df.merge(stores_df[['store_nbr', 'store_type_encoded']], on='store_nbr', how='left')


In [13]:
stores_df.head()

,store_nbr,city,state,type,cluster,store_type_encoded
0,1,Quito,Pichincha,D,13,3
1,2,Quito,Pichincha,D,13,3
2,3,Quito,Pichincha,D,8,3
3,4,Quito,Pichincha,D,9,3
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,3


In [15]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,day_of_week,store_type_encoded
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2013,1,1,1,3
1,1,2013-01-01,1,BABY CARE,0.0,0,2013,1,1,1,3
2,2,2013-01-01,1,BEAUTY,0.0,0,2013,1,1,1,3
3,3,2013-01-01,1,BEVERAGES,0.0,0,2013,1,1,1,3
4,4,2013-01-01,1,BOOKS,0.0,0,2013,1,1,1,3


In [62]:
#Création des features

In [22]:
import numpy as np

train_df["sales_log"] = np.log1p(train_df["sales"])  # log(1+sales) pour éviter log(0)


In [46]:

train_df["onpromotion"] = train_df["onpromotion"].astype(int)
train_df = train_df.sort_values(by=["store_nbr", "family", "date"]) 
train_df['duree_promotion'] = train_df.groupby(['store_nbr', 'family'])['onpromotion'].transform(lambda x: x.cumsum())
window_before = 7  # Jours avant la promotion
window_after = 7   # Jours après la promotion


In [50]:
train_df['sales_before_promotion'] = train_df.groupby(['store_nbr', 'family'])['sales'].shift(window_before)
train_df['sales_after_promotion'] = train_df.groupby(['store_nbr', 'family'])['sales'].shift(-window_after)
train_df['promotion_effect'] = train_df['sales_after_promotion'] - train_df['sales_before_promotion']

In [54]:
train_df['date'] = pd.to_datetime(train_df['date'])  # S'assurer que la colonne date est au bon format
train_df['month'] = train_df['date'].dt.month
train_df['season'] = train_df['month'].apply(lambda x: 'Spring' if 3 <= x <= 5 else 
                                       ('Summer' if 6 <= x <= 8 else 
                                        ('Fall' if 9 <= x <= 11 else 'Winter')))

In [64]:

train_df['promotion_season_interaction'] = train_df['onpromotion'] * train_df['season'].map({'Spring': 1, 'Summer': 2, 'Fall': 3, 'Winter': 4})

train_df['promotion_frequency'] = train_df.groupby(['store_nbr', 'family', 'month'])['onpromotion'].transform('sum')

In [58]:

print(train_df[['onpromotion', 'duree_promotion', 'sales_before_promotion', 'sales_after_promotion', 'promotion_effect', 'season', 'promotion_season_interaction', 'promotion_frequency']].head())

      onpromotion  duree_promotion  sales_before_promotion  \
0               0                0                     NaN   
1782            0                0                     NaN   
3564            0                0                     NaN   
5346            0                0                     NaN   
7128            0                0                     NaN   

      sales_after_promotion  promotion_effect  season  \
0                       2.0               NaN  Winter   
1782                    2.0               NaN  Winter   
3564                    2.0               NaN  Winter   
5346                    3.0               NaN  Winter   
7128                    2.0               NaN  Winter   

      promotion_season_interaction  promotion_frequency  
0                                0                    0  
1782                             0                    0  
3564                             0                    0  
5346                             0                  